In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-12-05 06:33:12--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.152.127.232, 52.44.92.122, 54.165.51.142, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.152.127.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  3.44MB/s    in 1.5s    

2018-12-05 06:33:14 (3.44 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


## Run TensorBoard

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

## Run ngrok

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

## Get URL
Run the next cell to start the training before open the url.

In [4]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://b403ebe9.ngrok.io


In [5]:
!unzip Daily.zip
!unzip Weekly.zip
!unzip FourHour.zip
!unzip OneHour.zip
!unzip ThirtyMin.zip
!unzip FifteenMin.zip

Archive:  Daily.zip
   creating: Daily/
   creating: Daily/Correction/
  inflating: Daily/Correction/XAUUSD_Correction0000.png  
  inflating: Daily/Correction/XAUUSD_Correction0001.png  
  inflating: Daily/Correction/XAUUSD_Correction0002.png  
  inflating: Daily/Correction/XAUUSD_Correction0003.png  
  inflating: Daily/Correction/XAUUSD_Correction0004.png  
  inflating: Daily/Correction/XAUUSD_Correction0005.png  
  inflating: Daily/Correction/XAUUSD_Correction0006.png  
  inflating: Daily/Correction/XAUUSD_Correction0007.png  
  inflating: Daily/Correction/XAUUSD_Correction0008.png  
  inflating: Daily/Correction/XAUUSD_Correction0009.png  
  inflating: Daily/Correction/XAUUSD_Correction0010.png  
  inflating: Daily/Correction/XAUUSD_Correction0011.png  
  inflating: Daily/Correction/XAUUSD_Correction0012.png  
  inflating: Daily/Correction/XAUUSD_Correction0013.png  
  inflating: Daily/Correction/XAUUSD_Correction0014.png  
  inflating: Daily/Correction/XAUUSD_Correction0015.png  
 

## Run a Keras model with TenorBoard

CNN

In [1]:
# DATA FEED IN AND PRE-PROCESSING -- CNN

import numpy as np
import cv2

height = 389
# weekly, daily, 4H, 1H, 30M, 15M
num_up = [6, 40, 97, 67, 123, 52]
num_down = [6, 40, 108, 62, 122, 47]
num_co = [84, 345, 0, 0, 0, 0]
timeframe = ['Weekly', 'Daily', 'FourHour', 'OneHour', 'ThirtyMin', 'FifteenMin']
TF = ['_weekly', '', '_4H', '_1H', '_30m', '_15m']
numbers = sum(num_up) + sum(num_down) + sum(num_co) #num_up + num_down + num_co #40+97-45+67 +40+108-45+62 +345
fresize = 0.4
height = int(height*fresize)+1

batch_size = 128
num_classes = 3

# input image dimensions
img_rows, img_cols = height, height

x_data = np.zeros((numbers, height, height, 1))
y_data = np.zeros(numbers)

count = 0
for i in range(len(num_up)):
  # up trend
  for j in range(num_up[i]):
    if i == 1 and j == 0:# Daily
        path = timeframe[i] + '/UpTrend/XAUUSD'+ TF[i] + '_Up' + '%04d'% j + '.PNG'
    else:
        path = timeframe[i] + '/UpTrend/XAUUSD'+ TF[i] + '_Up' + '%04d'% j + '.png'
    
    img = cv2.imread(path,0)
    if i < 2: # Weekly, Daily
        img = img[50:439, 1136-389:1136].astype('float32')
    else:
        img = img[50:439, 1178-389:1178].astype('float32')
      
    img = img/255
    img = cv2.resize(img, (0,0), fx=fresize, fy=fresize) 
    
    x_data[count,:,:,0] = img
    y_data[count] = y_data[count]
    count += 1
    
  # down trend
  for j in range(num_down[i]):
    if i == 1 and j == 0:# Daily
        path = timeframe[i] + '/DownTrend/XAUUSD'+ TF[i] + '_Down' + '%04d'% j + '.PNG'
    else:
        path = timeframe[i] + '/DownTrend/XAUUSD'+ TF[i] + '_Down' + '%04d'% j + '.png'
    
    img = cv2.imread(path,0)
    if i < 2: # Weekly, Daily
        img = img[50:439, 1136-389:1136].astype('float32')
    else:
        img = img[50:439, 1178-389:1178].astype('float32')
    
    img = img/255
    img = cv2.resize(img, (0,0), fx=fresize, fy=fresize) 
    
    x_data[count,:,:,0] = img
    y_data[count] = 1
    count += 1
    
  # correction
  for j in range(num_co[i]):
    path = timeframe[i] + '/Correction/XAUUSD'+ TF[i] + '_Correction' + '%04d'% j + '.png'
    
    img = cv2.imread(path,0)
    if i < 2: # Weekly, Daily
        img = img[50:439, 1136-389:1136].astype('float32')
    else:
        img = img[50:439, 1178-389:1178].astype('float32')
    
    img = img/255
    img = cv2.resize(img, (0,0), fx=fresize, fy=fresize) 
    
    x_data[count,:,:,0] = img
    y_data[count] = 2
    count += 1
    
    
# SHUFFLE TRAINING AND TESTING DATA

from sklearn import model_selection
import keras
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_data, y_data, test_size = 0.1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


In [2]:
# CNN


from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

epochs = 100

input_shape = (img_rows, img_cols, 1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.001),
              metrics=['accuracy'])


tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1079 samples, validate on 120 samples
Epoch 1/100
1079/1079 [==============================] - 9s 8ms/step - loss: 2.7219 - acc: 0.3207 - val_loss: 1.0975 - val_acc: 0.3333
Epoch 2/100
1079/1079 [==============================] - 4s 4ms/step - loss: 1.0961 - acc: 0.3503 - val_loss: 1.0941 - val_acc: 0.3417
Epoch 3/100
1079/1079 [==============================] - 4s 4ms/step - loss: 1.0776 - acc: 0.4282 - val_loss: 1.0841 - val_acc: 0.3917
Epoch 4/100
1079/1079 [==============================] - 4s 4ms/step - loss: 1.0643 - acc: 0.4384 - val_loss: 1.0711 - val_acc: 0.4000
Epoch 5/100
1079/1079 [==============================] - 4s 4ms/step - loss: 1.0427 - acc: 0.4949 - val_loss: 1.0446 - val_acc: 0.4667
Epoch 6/100
1079/1079 [==============================] - 4s 4ms/step - loss: 0.8950 - acc: 0.5941 - val_loss: 1.0188 - val_acc: 0.4917
Epoch 7/100
1079/1079 [==============================] - 4s 4ms/step - loss: 0.8307 - acc: 0.6321 - val_loss: 0.9718 - val_acc: 0.5667
Epoch 8/

In [33]:
[sum(num_up),  sum(num_down),  sum(num_no)]

[385, 385, 429]

In [42]:
np.shape(x_train), np.shape(x_test), np.shape(y_train), np.shape(y_test)

((1079, 156, 156), (120, 156, 156), (1079,), (120,))

RNN

In [0]:
# DATA FEED IN AND PRE-PROCESSING -- RNN

import numpy as np
import cv2

height = 389
# weekly, daily, 4H, 1H, 30M, 15M
num_up = [6, 40, 97, 67, 123, 52]
num_down = [6, 40, 108, 62, 122, 47]
num_co = [84, 345, 0, 0, 0, 0]
timeframe = ['Weekly', 'Daily', 'FourHour', 'OneHour', 'ThirtyMin', 'FifteenMin']
TF = ['_weekly', '', '_4H', '_1H', '_30m', '_15m']
numbers = sum(num_up) + sum(num_down) + sum(num_co) #num_up + num_down + num_co #40+97-45+67 +40+108-45+62 +345
fresize = 0.4
height = int(height*fresize)+1

batch_size = 128
num_classes = 3

# input image dimensions
img_rows, img_cols = height, height

x_data = np.zeros((numbers, height, height))
y_data = np.zeros(numbers)

count = 0
for i in range(len(num_up)):
  # up trend
  for j in range(num_up[i]):
    if i == 1 and j == 0:# Daily
        path = timeframe[i] + '/UpTrend/XAUUSD'+ TF[i] + '_Up' + '%04d'% j + '.PNG'
    else:
        path = timeframe[i] + '/UpTrend/XAUUSD'+ TF[i] + '_Up' + '%04d'% j + '.png'
    
    img = cv2.imread(path,0)
    if i < 2: # Weekly, Daily
        img = img[50:439, 1136-389:1136].astype('float32')
    else:
        img = img[50:439, 1178-389:1178].astype('float32')
    
    img = img/255
    img = cv2.resize(img, (0,0), fx=fresize, fy=fresize) 
    
    x_data[count,:,:] = img
    y_data[count] = y_data[count]
    count += 1
    
  # down trend
  for j in range(num_down[i]):
    if i == 1 and j == 0:# Daily
        path = timeframe[i] + '/DownTrend/XAUUSD'+ TF[i] + '_Down' + '%04d'% j + '.PNG'
    else:
        path = timeframe[i] + '/DownTrend/XAUUSD'+ TF[i] + '_Down' + '%04d'% j + '.png'
    
    img = cv2.imread(path,0)
    if i < 2: # Weekly, Daily
        img = img[50:439, 1136-389:1136].astype('float32')
    else:
        img = img[50:439, 1178-389:1178].astype('float32')
    
    img = img/255
    img = cv2.resize(img, (0,0), fx=fresize, fy=fresize) 
    
    x_data[count,:,:] = img
    y_data[count] = 1
    count += 1
    
  # correction
  for j in range(num_co[i]):
    path = timeframe[i] + '/Correction/XAUUSD'+ TF[i] + '_Correction' + '%04d'% j + '.png'
    
    img = cv2.imread(path,0)
    if i < 2: # Weekly, Daily
        img = img[50:439, 1136-389:1136].astype('float32')
    else:
        img = img[50:439, 1178-389:1178].astype('float32')
    
    img = img/255
    img = cv2.resize(img, (0,0), fx=fresize, fy=fresize) 
    
    x_data[count,:,:] = img
    y_data[count] = 2
    count += 1
    
    
# SHUFFLE TRAINING AND TESTING DATA

from sklearn import model_selection
import keras
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_data, y_data, test_size = 0.1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
import tensorflow as tf
import os
from tensorflow.python.ops import rnn, rnn_cell

In [7]:
# LSTM 

learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200
test_step = 1000

nInput = height 
nSteps = height 
nHidden = 128  
nClasses = 3
result_dir = './results_LSTM/'

x = tf.placeholder('float', [None, nSteps, nInput])
y = tf.placeholder('float', [None, nClasses])

weights = {
    'out': tf.Variable(tf.random_normal([nHidden, nClasses]))
}

biases = {
    'out': tf.Variable(tf.random_normal([nClasses]))
}


def RNN(x, weights, biases):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, nInput])
    x = tf.split(x, nSteps, 0)
    lstm_cell = tf.nn.rnn_cell.LSTMCell(nHidden, forget_bias=1.0)#, reuse=True)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']


def unfoldRNN(x):
    return tf.unstack(x, nSteps, 1)


pred = RNN(x, weights, biases)
prediction = tf.nn.softmax(pred)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model (with test logits, for dropout to be disabled)
correctPred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss_summary = tf.summary.scalar("loss", cost)
accuracy_summary = tf.summary.scalar("accuracy", accuracy)
test_accuracy_summary = tf.summary.scalar("test_accuracy", accuracy)

# Initialize the variables (i.e. assign their default value)
init = tf.initialize_all_variables()

# Start training

with tf.Session() as sess:
    saver = tf.train.Saver()

    summary_writer = tf.summary.FileWriter(result_dir, sess.graph)
    sess.run(init)

    for step in range(1, training_steps + 1):
        shuffles_train = list(range(len(x_train)))
        np.random.shuffle(shuffles_train)
        batchX = x_train[shuffles_train[:batch_size],:,:]
        batchY = y_train[shuffles_train[:batch_size]]
        sess.run(optimizer, feed_dict={x: batchX, y: batchY})

        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc, loss_summ, accuracy_summ = sess.run([cost, accuracy, loss_summary, accuracy_summary],
                                                           feed_dict={x: batchX,
                                                                      y: batchY})
            summary_writer.add_summary(loss_summ, step)
            summary_writer.add_summary(accuracy_summ, step)
            summary_writer.flush()

            print("Training Step " + str(step) + ", Minibatch Loss= " +
                  "{:.6f}".format(loss) + ", Training Accuracy= " +
                  "{:.5f}".format(acc))

        if step % test_step == 0 or step == 1:
            shuffles_test = list(range(len(x_test)))
            np.random.shuffle(shuffles_test)
            loss, acc, loss_summ, test_accuracy_summ = sess.run([cost, accuracy, loss_summary, test_accuracy_summary],
                                                                feed_dict={
                                                                    x: (x_test[shuffles_test[:batch_size],:,:]),
                                                                    y: y_test[shuffles_test[:batch_size]]})
            summary_writer.add_summary(test_accuracy_summ, step)
            checkpoint_file = os.path.join(result_dir, 'checkpoint')
            saver.save(sess, checkpoint_file, global_step=step)
            summary_writer.flush()

            print("Test Step " + str(step) + ", Minibatch Loss= " +
                  "{:.6f}".format(loss) + ", Testing Accuracy= " +
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    testData = x_test[:,:,:]
    testLabel = y_test[:]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: testData, y: testLabel}))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Training Step 1, Minibatch Loss= 1.136090, Training Accuracy= 0.39062
Test Step 1, Minibatch Loss= 1.198355, Testing Accuracy= 0.30000
Training Step 200, Minibatch Loss= 1.023054, Training Accuracy= 0.59375
Training Step 400, Minibatch Loss= 0.958115, Training Accuracy= 0.57031
Training Step 600, Minibatch Loss= 0.873392, Training Accuracy= 0.64844
Training Step 800, Minibatch Loss= 0.889890, Training Accuracy= 0.61719
Training Step 1000, Minibatch Loss= 0.851353, Training Accuracy= 0.61719
Test Step 1000, Minibatch Loss= 0.838344, Testing Accuracy= 0.64167
Training Step 1200, Minibatch Loss= 0.885591, Training Accuracy= 0.53125
Training Step 1400, Minibatch Loss= 0.880929, Training Accuracy= 0.58594
Training Step 1600, Minib

In [5]:

import numpy as np
np.shape(batchX)

(128, 156, 156)

In [8]:
# GRU


learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200
test_step = 1000

nInput = height 
nSteps = height  
nHidden = 128 
nClasses = 3
result_dir = './results_GRU/'

x = tf.placeholder('float', [None, nSteps, nInput])
y = tf.placeholder('float', [None, nClasses])

weights = {
    'out': tf.Variable(tf.random_normal([nHidden, nClasses]))
}

biases = {
    'out': tf.Variable(tf.random_normal([nClasses]))
}


def RNN(x, weights, biases):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, nInput])
    x = tf.split(x, nSteps, 0)
    gru_cell = rnn_cell.GRUCell(nHidden)
    outputs, states = rnn.static_rnn(gru_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']


def unfoldRNN(x):
    return tf.unstack(x, nSteps, 1)


pred = RNN(x, weights, biases)
prediction = tf.nn.softmax(pred)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model (with test logits, for dropout to be disabled)
correctPred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss_summary = tf.summary.scalar("loss", cost)
accuracy_summary = tf.summary.scalar("accuracy", accuracy)
test_accuracy_summary = tf.summary.scalar("test_accuracy", accuracy)

# Initialize the variables (i.e. assign their default value)
init = tf.initialize_all_variables()

# Start training

with tf.Session() as sess:
    saver = tf.train.Saver()

    summary_writer = tf.summary.FileWriter(result_dir, sess.graph)
    sess.run(init)

    for step in range(1, training_steps + 1):
        shuffles_train = list(range(len(x_train)))
        np.random.shuffle(shuffles_train)
        batchX = x_train[shuffles_train[:batch_size],:,:]
        batchY = y_train[shuffles_train[:batch_size]]
        sess.run(optimizer, feed_dict={x: batchX, y: batchY})

        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc, loss_summ, accuracy_summ = sess.run([cost, accuracy, loss_summary, accuracy_summary],
                                                           feed_dict={x: batchX,
                                                                      y: batchY})
            summary_writer.add_summary(loss_summ, step)
            summary_writer.add_summary(accuracy_summ, step)
            summary_writer.flush()

            print("Training Step " + str(step) + ", Minibatch Loss= " +
                  "{:.6f}".format(loss) + ", Training Accuracy= " +
                  "{:.5f}".format(acc))

        if step % test_step == 0 or step == 1:
            shuffles_test = list(range(len(x_test)))
            np.random.shuffle(shuffles_test)
            loss, acc, loss_summ, test_accuracy_summ = sess.run([cost, accuracy, loss_summary, test_accuracy_summary],
                                                                feed_dict={
                                                                    x: (x_test[shuffles_test[:batch_size],:,:]),
                                                                    y: y_test[shuffles_test[:batch_size]]})
            summary_writer.add_summary(test_accuracy_summ, step)
            checkpoint_file = os.path.join(result_dir, 'checkpoint')
            saver.save(sess, checkpoint_file, global_step=step)
            summary_writer.flush()

            print("Test Step " + str(step) + ", Minibatch Loss= " +
                  "{:.6f}".format(loss) + ", Testing Accuracy= " +
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    testData = x_test[:,:,:]
    testLabel = y_test[:]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: testData, y: testLabel}))


Training Step 1, Minibatch Loss= 1.570676, Training Accuracy= 0.36719
Test Step 1, Minibatch Loss= 1.579638, Testing Accuracy= 0.32500
Training Step 200, Minibatch Loss= 1.003041, Training Accuracy= 0.56250
Training Step 400, Minibatch Loss= 0.940247, Training Accuracy= 0.40625
Training Step 600, Minibatch Loss= 0.906239, Training Accuracy= 0.53125
Training Step 800, Minibatch Loss= 0.858382, Training Accuracy= 0.54688
Training Step 1000, Minibatch Loss= 0.835003, Training Accuracy= 0.64844
Test Step 1000, Minibatch Loss= 0.856358, Testing Accuracy= 0.56667
Training Step 1200, Minibatch Loss= 0.823104, Training Accuracy= 0.63281
Training Step 1400, Minibatch Loss= 0.820316, Training Accuracy= 0.59375
Training Step 1600, Minibatch Loss= 0.857112, Training Accuracy= 0.48438
Training Step 1800, Minibatch Loss= 0.811211, Training Accuracy= 0.58594
Training Step 2000, Minibatch Loss= 0.778678, Training Accuracy= 0.59375
Test Step 2000, Minibatch Loss= 0.765318, Testing Accuracy= 0.57500
Tra

In [9]:
# RNN


learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200
test_step = 1000

nInput = height 
nSteps = height 
nHidden = 128  
nClasses = 3
result_dir = './results_RNN/'

x = tf.placeholder('float', [None, nSteps, nInput])
y = tf.placeholder('float', [None, nClasses])

weights = {
    'out': tf.Variable(tf.random_normal([nHidden, nClasses]))
}

biases = {
    'out': tf.Variable(tf.random_normal([nClasses]))
}


def RNN(x, weights, biases):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, nInput])
    x = tf.split(x, nSteps, 0)
    RNN_cell = rnn_cell.BasicRNNCell(nHidden)
    outputs, states = rnn.static_rnn(RNN_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']


def unfoldRNN(x):
    return tf.unstack(x, nSteps, 1)


pred = RNN(x, weights, biases)
prediction = tf.nn.softmax(pred)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model (with test logits, for dropout to be disabled)
correctPred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss_summary = tf.summary.scalar("loss", cost)
accuracy_summary = tf.summary.scalar("accuracy", accuracy)
test_accuracy_summary = tf.summary.scalar("test_accuracy", accuracy)

# Initialize the variables (i.e. assign their default value)
init = tf.initialize_all_variables()

# Start training

with tf.Session() as sess:
    saver = tf.train.Saver()

    summary_writer = tf.summary.FileWriter(result_dir, sess.graph)
    sess.run(init)

    for step in range(1, training_steps + 1):
        shuffles_train = list(range(len(x_train)))
        np.random.shuffle(shuffles_train)
        batchX = x_train[shuffles_train[:batch_size],:,:]
        batchY = y_train[shuffles_train[:batch_size]]
        sess.run(optimizer, feed_dict={x: batchX, y: batchY})

        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc, loss_summ, accuracy_summ = sess.run([cost, accuracy, loss_summary, accuracy_summary],
                                                           feed_dict={x: batchX,
                                                                      y: batchY})
            summary_writer.add_summary(loss_summ, step)
            summary_writer.add_summary(accuracy_summ, step)
            summary_writer.flush()

            print("Training Step " + str(step) + ", Minibatch Loss= " +
                  "{:.6f}".format(loss) + ", Training Accuracy= " +
                  "{:.5f}".format(acc))

        if step % test_step == 0 or step == 1:
            shuffles_test = list(range(len(x_test)))
            np.random.shuffle(shuffles_test)
            loss, acc, loss_summ, test_accuracy_summ = sess.run([cost, accuracy, loss_summary, test_accuracy_summary],
                                                                feed_dict={
                                                                    x: (x_test[shuffles_test[:batch_size],:,:]),
                                                                    y: y_test[shuffles_test[:batch_size]]})
            summary_writer.add_summary(test_accuracy_summ, step)
            checkpoint_file = os.path.join(result_dir, 'checkpoint')
            saver.save(sess, checkpoint_file, global_step=step)
            summary_writer.flush()

            print("Test Step " + str(step) + ", Minibatch Loss= " +
                  "{:.6f}".format(loss) + ", Testing Accuracy= " +
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    testData = x_test[:,:,:]
    testLabel = y_test[:]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: testData, y: testLabel}))


Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Training Step 1, Minibatch Loss= 2.844236, Training Accuracy= 0.31250
Test Step 1, Minibatch Loss= 2.542162, Testing Accuracy= 0.37500
Training Step 200, Minibatch Loss= 0.926534, Training Accuracy= 0.53906
Training Step 400, Minibatch Loss= 0.754711, Training Accuracy= 0.62500
Training Step 600, Minibatch Loss= 0.770410, Training Accuracy= 0.63281
Training Step 800, Minibatch Loss= 0.812265, Training Accuracy= 0.57031
Training Step 1000, Minibatch Loss= 0.784910, Training Accuracy= 0.63281
Test Step 1000, Minibatch Loss= 0.775242, Testing Accuracy= 0.56667
Training Step 1200, Minibatch Loss= 0.822709, Training Accuracy= 0.59375
Training Step 1400, Minibatch Loss= 0.793107, Training Accuracy= 0.58594
Training Step 1600, Minibatch Loss= 0.746164, Training Accuracy= 0.60938
Training Step 1800, Minibatch Loss= 0.814168, Training Accuracy= 0.57812
Training S